In [6]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, pprint, time, re
from urllib import parse

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--incognito')
options.add_argument('--disable-popup-blocking')

driver = webdriver.Chrome(options = options)

listData = []
url = 'https://www.gutenberg.org/browse/languages/zh'

def visit(): # 請瀏覽器到這個網址去
    driver.get(url)
    
def getMainLinks(): # 取得所有主要的連結  # 用 for 改 title 名字太麻煩
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'li.pgdbetext > a')
    
    for a in a_elms:

        title = a.get_attribute('innerText')
        title = title.replace(" ", "")
        title = title.replace("\r", "")
        title = title.replace("\n", "")
        title = title.replace("<", "")
        title = title.replace(">", "")
        title = title.replace("/", "")
        title = title.replace("\\", "")
        title = title.replace("*", "")
        title = title.replace("|", "")
        title = title.replace("\"", "")

        listData.append({
            "title": title,
            "link":a.get_attribute('href') + '.html.noimages'
        })
#     pprint.pprint(listData)
    
def get_Link():
    for i in range( len(listData) ):
        driver.get(listData[i]["link"])
            

def saveJson():
    fp = open("sel_gutenberg.json", "w", encoding="utf-8")
    fp.write( json.dumps(listData, ensure_ascii=False))
    fp.close()
    
def writeTxt():
    # listContent = []
    fp = open("sel_gutenberg.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()

    folderPath = "sel_gutenberg_txt"
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
        
    listResult = json.loads(strJson)
    
    for i in range( len(listResult) ):
        driver.get(listResult[i]["link"]) # 各別走訪 瀏覽器進去裡面走訪
        body = driver.find_element(By.CSS_SELECTOR, 'body')

        strContent = body.get_attribute('innerText')
        regexContent = r"[\u4E00-\u9FFF，。：「」；、？！『』]+"
        strContent = "".join(re.findall(regexContent, strContent))

        fileName = f"{listResult[i]['title']}.txt"
        fp = open(f"{folderPath}/{fileName}", "w", encoding="utf-8")
        fp.write( strContent )
        fp.close()
    
def close():
    driver.quit()
    
if __name__ == "__main__":
    visit()
    getMainLinks()
    get_Link()
    saveJson()
    writeTxt()
    close()